## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# importing sys
import sys
  
# adding parent directory to the system path so the config.py is accessible
sys.path.insert(0, '../')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Pevek,RU,69.7008,170.3133,-30.62,99,65,4.76,broken clouds
1,Jamestown,US,42.0970,-79.2353,36.72,94,90,3.00,mist
2,Faya,SA,18.3851,42.4509,59.52,71,2,1.72,clear sky
3,Norman Wells,CA,65.2820,-126.8329,-2.16,78,90,0.00,light snow
4,Saint-Philippe,RE,-21.3585,55.7679,74.79,83,89,8.63,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temperature = float(input(f'Please enter the minimum temperature in Fahrenheit for your dream vacation: '))
max_temperature = float(input(f'Please enter the maximum temperature in Fahrenheit for your dream vacation: '))

Please enter the minimum temperature in Fahrenheit for your dream vacation: 14
Please enter the maximum temperature in Fahrenheit for your dream vacation: 50


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = \
city_data_df.loc[(city_data_df['Max Temp'] <= max_temperature) & \
                 (city_data_df['Max Temp'] >= min_temperature)]

In [5]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City                   139
Country                139
Lat                    139
Lng                    139
Max Temp               139
Humidity               139
Cloudiness             139
Wind Speed             139
Current Description    139
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Jamestown,US,36.72,mist,42.0970,-79.2353,
5,Khonsa,IN,49.42,broken clouds,27.0167,95.5667,
6,Albany,US,33.10,overcast clouds,42.6001,-73.9662,
7,Severo-Kurilsk,RU,31.37,light snow,50.6789,156.1250,
9,Ushuaia,AR,49.62,light rain,-54.8000,-68.3000,
10,Paamiut,GL,17.82,clear sky,61.9940,-49.6678,
23,Serebryansk,KZ,18.95,overcast clouds,49.6925,83.2892,
35,Saint-Augustin,CA,35.40,overcast clouds,51.2260,-58.6502,
38,Klaksvik,FO,41.92,overcast clouds,62.2266,-6.5890,
49,Preston,GB,48.33,overcast clouds,53.7667,-2.7167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    city_lat = row['Lat']
    city_lng = row['Lng']
    
    # Add the latitude and longitude to location key for the params dictionary
    params['location'] = f'{city_lat},{city_lng}'

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
#        print(hotels['results'][0]['name'])
        
    except(IndexError):
        print('Hotel not found... skipping.')
        hotel_df.loc[index, 'Hotel Name'] = ''     
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# Checking empty hotel names
hotel_df.loc[hotel_df['Hotel Name'] == '']

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Khonsa,IN,49.42,broken clouds,27.0167,95.5667,
6,Albany,US,33.10,overcast clouds,42.6001,-73.9662,
35,Saint-Augustin,CA,35.40,overcast clouds,51.2260,-58.6502,
86,Komyshuvakha,UA,22.14,overcast clouds,47.7176,35.5288,
106,Dzhalka,RU,32.88,broken clouds,43.3186,45.9879,
235,Yumen,CN,16.43,overcast clouds,40.2833,97.2000,
236,Sobolevo,RU,14.25,broken clouds,54.4299,31.8996,
282,Bereznehuvate,UA,26.74,snow,47.3078,32.8499,
353,Teguldet,RU,15.66,light snow,57.3064,88.1667,
425,Baherden,TM,42.48,scattered clouds,38.4362,57.4316,


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df.loc[hotel_df['Hotel Name'] == '' ].index)

In [11]:
# Check data frame
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Jamestown,US,36.72,mist,42.0970,-79.2353,Hampton Inn & Suites Jamestown
7,Severo-Kurilsk,RU,31.37,light snow,50.6789,156.1250,Kak Doma
9,Ushuaia,AR,49.62,light rain,-54.8000,-68.3000,Albatros Hotel
10,Paamiut,GL,17.82,clear sky,61.9940,-49.6678,Pistut
23,Serebryansk,KZ,18.95,overcast clouds,49.6925,83.2892,"Gostinitsa ""Vodoley"""


In [12]:
# 8a. Create the output File (CSV)
output_data_file = '../Vacation_Search/WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt>{Current Description} and {Max Temp} F<dd></dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(0,0), zoom_level = 3)
heat_layer = gmaps.heatmap_layer(locations, weights = clean_hotel_df['Max Temp'],
                                 dissipating = False, max_intensity = 300, point_radius = 4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Saving the map as a .html in the disk
from ipywidgets.embed import embed_minimal_html
embed_minimal_html('../Vacation_Search/WeatherPy_vacation_map.html', views=[fig])